In [1]:
#importing basic libraries
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#implicit is the library used for collaborative filtering in recommendation systems
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 20.1 MB/s eta 0:00:00


In [3]:
import implicit

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [6]:
#loading data
data= pd.read_excel("Book Report-vistaar.xlsx")
data.head(5)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,User,Book,Total Time Spent on Book (in mins),Total No of Pages Read,Most Read Page
0,AS,AquaDE,83.3999,13,aquacs
1,AK,AquaDE,4.5333,37,aquaft
2,AK,AquaDE,7.3500,68,aquacs
3,AL,AquaDE,89.7166,640,aquacs
4,AC,AquaDE,16.6000,33,aquaft


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1242 entries, 0 to 1241
Data columns (total 5 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   User                                 1219 non-null   object 
 1   Book                                 1242 non-null   object 
 2   Total Time Spent on Book (in mins)   1242 non-null   float64
 3   Total No of Pages Read               1242 non-null   int64  
 4   Most Read Page                       1242 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 48.6+ KB


In [8]:
import numpy as np
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

In [9]:
#made a dictionary to give each user and book a user id and book id to make calculations easier
unique_user=data["User"].unique()
User_ids = dict(zip(unique_user, np.arange(unique_user.shape[0], dtype=np.int32)))
print(User_ids)


{'AS': 0, 'AK': 1, 'AL': 2, 'AC': 3, 'AM': 4, 'DKA': 5, 'DA': 6, 'GHG': 7, 'HS': 8, 'JSK': 9, 'KM': 10, 'KSA': 11, 'KAK': 12, 'KV': 13, 'MS': 14, 'MKC': 15, 'MA': 16, 'MKB': 17, nan: 18, 'NV': 19, 'PK': 20, 'PS': 21, 'PM': 22, 'RSA': 23, 'RS': 24, 'RPSG': 25, 'SMA': 26, 'SVR': 27, 'SS': 28, 'SM': 29, 'SSB': 30, 'SD': 31, 'SKM': 32, 'SSK': 33, 'SB': 34, 'TKY': 35, 'TNK': 36, 'US': 37, 'VSk': 38, 'VB': 39, 'VKGM': 40, 'VS': 41, 'VH': 42, 'WK': 43}


In [10]:
unique_books = data['Book'].unique()
Book_ids = dict(zip(unique_books, np.arange(unique_books.shape[0], dtype=np.int32)))

In [11]:
data['User_ids'] = data["User"].apply(lambda i: User_ids[i])
data['Book_ids'] = data['Book'].apply(lambda i: Book_ids[i])

In [12]:
import scipy.sparse as sparse

In [13]:
# formed matries between user,book and pages they read this will help to understand user's interaction with each book
sparse_book_user = sparse.csr_matrix((data['Total No of Pages Read'].astype(float), (data['Book_ids'], data['User_ids'])))
sparse_user_book = sparse.csr_matrix((data['Total No of Pages Read'].astype(float), (data['User_ids'], data['Book_ids'])))

In [14]:
# AlternatingLeastSquares is used to get latent factors of the interactions
model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=10)

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [15]:
alpha = 15
df = (sparse_book_user * alpha).astype('double')

model.fit(df)

  0%|          | 0/10 [00:00<?, ?it/s]

In [16]:
item_vecs = model.item_factors
user_vecs = model.user_factors


In [17]:
from sklearn.preprocessing import MinMaxScaler

In the recommend function:
1. the user interactions from the above matrix are converted to 1D array
2. similarly the latent factors from ALS are converted array
3. both are multiplied to produce a score upon which each book will be judged
4. the system recommends 5 books, considering the fact that the user may have completed the books whose many pages are already read or something not to be recommended.
5. for book recommended the system provides an approx time they may need to read avgerage amount of pages in the book

In [20]:
def recommend(User_ids, sparse_user_book, user_vecs, item_vecs, num_items=19):

    user_interactions = sparse_user_book[User_ids,:].toarray()
    user_interactions = user_interactions.reshape(-1)+1


    rec_vector = user_vecs[User_ids,:].dot(item_vecs.T).toarray()

    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    common_indices = np.intersect1d(np.arange(user_interactions.shape[0]), np.arange(rec_vector_scaled.shape[0]))
    user_interactions = user_interactions[common_indices]
    rec_vector_scaled = rec_vector_scaled[common_indices]
    recommend_vector = user_interactions * rec_vector_scaled

    item_idx = np.argsort(recommend_vector)[::-1][num_items:]


    books = []
    scores = []
    avg_time_book=[]


    for idx in item_idx:
        books.append(data["Book"].loc[data["Book_ids"] == idx].iloc[0])
        scores.append(recommend_vector[idx])
        avg_time=data["Total Time Spent on Book (in mins) "].loc[data['Book_ids']==idx].mean()
        avg_time_page=data["Total No of Pages Read"].loc[data['Book_ids']==idx].mean()
        avg_time_book.append(avg_time*avg_time_page)


    recommendations = pd.DataFrame({'Book': books, 'score': scores,'Time to taken(in mins)': avg_time_book})

    return recommendations

In [22]:
user_vecs = sparse.csr_matrix(model.user_factors)
item_vecs = sparse.csr_matrix(model.item_factors)

# Create recommendations for customer with id 2
print("Get recommendation for top 5 books and time you will take on avgerage to read them \n")
a=input("Enter user name\n")
User_id=User_ids.get(a)

recommendations = recommend(User_id, sparse_user_book, user_vecs, item_vecs)

print(recommendations)

Get recommendation for top 5 books and time you will take on avgerage to read them 

Enter user name
AL
      Book      score  Time to taken(in mins)
0   LoteCJ  19.917150             1052.595480
1      CBR   9.967759               24.644234
2  BrinzGL   3.997900              829.859242
3  GaviOHP   2.959559              384.661221
4   NetaGL   0.997497             1026.344976
